<a href="https://colab.research.google.com/github/ryannhally/Honors_Project_And_Essay/blob/main/NAH_Morfessor_w_bleu_formula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install libraries
!pip install transformers datasets evaluate sacrebleu morfessor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 require

In [ ]:
# Import mofessor
import morfessor

In [ ]:
# Import google drive
from google.colab import drive

In [ ]:
# Mount drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Output location
output_path = '/content/drive/MyDrive/morfessor_8.bin'

In [ ]:
# Read file
io = morfessor.MorfessorIO()
model = io.read_binary_model_file(output_path)
print(type(model))

<class 'morfessor.baseline.BaselineModel'>


In [ ]:
# Segment morphemes
def get_morphemes(translation):
  words = translation.split()
  morphemes = []
  for word in words:
    extracted_morphemes = model.segment(word)
    #print("WORD: ", word)
    #print("MORPHEMES: ", extracted_morphemes)
    morphemes += extracted_morphemes
  #returns every morpheme in translation
  return morphemes


In [ ]:
# Extract n-grams
from collections import Counter

def get_ngrams(morphemes_list):
  frequencies = Counter(morphemes_list)
  return frequencies


# Calculate score, blue implemented by me

In [ ]:
# Calculate precision - no brevity penalty
def calculate_precision(reference_ngrams, candidate_ngrams):

  correct_predictions = 0

  # Number of ngrams in candidate
  sum_morphemes_candidate = sum(candidate_ngrams.values())


  for candidate_morpheme in candidate_ngrams:
    if candidate_morpheme in reference_ngrams:
      if candidate_ngrams[candidate_morpheme] > reference_ngrams[candidate_morpheme]:
        correct_predictions += reference_ngrams[candidate_morpheme]
      else:
        correct_predictions += candidate_ngrams[candidate_morpheme]

  # Calculate precision
  precision = correct_predictions / sum_morphemes_candidate


  # Check value

  print("         REREFERENCE NGRAMS:", reference_ngrams)
  print("         CANDIDATE NGRAMS:", candidate_ngrams)
  print("         CORRECT PREDICTIONS: ", correct_predictions)
  print("         SUM MORPHEMES CANDIDATE: ", sum_morphemes_candidate)
  #print("         PRECISION: ", precision)


  return precision



In [ ]:
# Check new formula !
print(evaluate("quenin mochihua", "kenin mochiua"))

REREFERENCE NGRAMS: Counter({'quen': 1, 'in': 1, 'mo': 1, 'chihua': 1})
CANDIDATE NGRAMS: Counter({'ke': 1, 'nin': 1, 'mo': 1, 'chiua': 1})
CORRECT PREDICTIONS:  1
SUM MORPHEMES CANDIDATE:  4
PRECISION:  0.25
0.25


In [ ]:
# Calculate precision - WITH brevity penalty
import math

def calculate_precision_penalty(reference_ngrams, candidate_ngrams):

  correct_predictions = 0

  # Number of ngrams in candidate
  sum_morphemes_candidate = sum(candidate_ngrams.values())


  for candidate_morpheme in candidate_ngrams:
    if candidate_morpheme in reference_ngrams:
      if candidate_ngrams[candidate_morpheme] > reference_ngrams[candidate_morpheme]:
        correct_predictions += reference_ngrams[candidate_morpheme]
      else:
        correct_predictions += candidate_ngrams[candidate_morpheme]

  # Calculate precision
  precision = correct_predictions / sum_morphemes_candidate

  # Brevity penalty
  c = len(candidate_ngrams)
  r = len(reference_ngrams)

  #print("R: ", r)
  #print("C: ", c)

  if c >= r :
    brevity_penalty = 1

  else:
    brevity_penalty = math.exp((1-r)/c)

  #print("    Brevity penalty: ", brevity_penalty)

  precision_w_penalty = brevity_penalty * precision

  print("         REREFERENCE NGRAMS:", reference_ngrams)
  print("         CANDIDATE NGRAMS:", candidate_ngrams)
  print("         CORRECT PREDICTIONS: ", correct_predictions)
  print("         SUM MORPHEMES CANDIDATE: ", sum_morphemes_candidate)


  return precision_w_penalty

In [ ]:
# Calculate score (my implementation, no brevity penalty)

def evaluate(reference, candidate):
  reference_morphemes = get_morphemes(reference)
  canidate_morphemes = get_morphemes(candidate)

  reference_ngrams = get_ngrams(reference_morphemes)
  candidate_grams = get_ngrams(canidate_morphemes)

  score = calculate_precision(reference_ngrams, candidate_grams)

  #print("SCORE: ", score)
  return score


In [ ]:
# Calculate score (my implementation, WITH brevity penalty)

def evaluate_penalty(reference, candidate):
  reference_morphemes = get_morphemes(reference)
  canidate_morphemes = get_morphemes(candidate)

  reference_ngrams = get_ngrams(reference_morphemes)
  candidate_grams = get_ngrams(canidate_morphemes)

  score = calculate_precision_penalty(reference_ngrams, candidate_grams)

  #print("SCORE: ", score)
  return score


In [ ]:
# Calculate score (nltk bleu)


# Test formulas

In [ ]:
# Test no brevity penalty
reference = 'maxalihuiliztli'
candidate_perfect = 'maxalihuiliztli'
candidate_wrong = 'maxalihui'
#candidate = 'tlen motekiuiya'
print(evaluate(reference, candidate_perfect))
print(evaluate(reference, candidate_wrong))

Sum candidate ngrams:  2
1.0
Sum candidate ngrams:  2
0.5


In [ ]:
# Test brevity penality

reference = 'maxalihuiliztli'
# same length, brevity penalty should equal 1
candidate_perfect = 'maxalihuiliztli'
# candidate short, brevity penalty should be .36....
candidate_short = 'maxalihui'
# candidate long, brevity penalty should equal 1
print(evaluate_penalty(reference, candidate_perfect))
print(evaluate_penalty(reference, candidate_short))
print(evaluate_penalty(candidate_short, reference))


   Sum candidate ngrams:  2
   Brevity penalty:  1
1.0
   Sum candidate ngrams:  2
   Brevity penalty:  0.36787944117144233
0.18393972058572117
   Sum candidate ngrams:  1
   Brevity penalty:  1
1.0


In [ ]:
# Evaluate using my bleu implementation without brevity penalty # SOMETHING WRONG WITH SUM CANDIDATE N GRAMS
import re

file_path = 'refs_and_canids.txt'
count = 1
candidates = []


with open(file_path, 'r') as file:
  lines = file.readlines()

  # in groups of 5
  for i in range(0, len(lines), 5):
    # saves first line as reference
    reference = lines[i].strip().lower()  # First line is reference
    print("REFERENCE: ", reference)

    # save lines 2-4 as candidates
    for j in range(i + 1, i + 5):
      candidates += [lines[j].strip().lower()]


    for k, candidate in enumerate(candidates):
        print(f"     translation {k + 1}: ", candidate)
        score = evaluate(reference, candidate)
        print("     score:", score)

    candidates = []

REFERENCE:  itlanahnamic
     translation 1:  tlen motekiuiya
         REREFERENCE NGRAMS: Counter({'i': 1, 'tla': 1, 'nah': 1, 'namic': 1})
         CANDIDATE NGRAMS: Counter({'tlen': 1, 'mo': 1, 'te': 1, 'ki': 1, 'ui': 1, 'ya': 1})
         CORRECT PREDICTIONS:  0
         SUM MORPHEMES CANDIDATE:  6
     score: 0.0
     translation 2:  itlanajnamik
         REREFERENCE NGRAMS: Counter({'i': 1, 'tla': 1, 'nah': 1, 'namic': 1})
         CANDIDATE NGRAMS: Counter({'i': 1, 'tlan': 1, 'ajnamik': 1})
         CORRECT PREDICTIONS:  1
         SUM MORPHEMES CANDIDATE:  3
     score: 0.3333333333333333
     translation 3:  itlanajnamik
         REREFERENCE NGRAMS: Counter({'i': 1, 'tla': 1, 'nah': 1, 'namic': 1})
         CANDIDATE NGRAMS: Counter({'i': 1, 'tlan': 1, 'ajnamik': 1})
         CORRECT PREDICTIONS:  1
         SUM MORPHEMES CANDIDATE:  3
     score: 0.3333333333333333
     translation 4:  itlanajnamik
         REREFERENCE NGRAMS: Counter({'i': 1, 'tla': 1, 'nah': 1, 'namic': 1})

In [ ]:
# Evaluate using my bleu implementation WITH brevity penalty
import re

file = 'refs_and_canids.txt'
count = 1
candidates = []


with open(file_path, 'r') as file:
  lines = file.readlines()

  # in groups of 5
  for i in range(0, len(lines), 5):
    # saves first line as reference
    reference = lines[i].strip().lower()  # First line is reference
    print("REFERENCE: ", reference)

    # save lines 2-4 as candidates
    for j in range(i + 1, i + 5):
      candidates += [lines[j].strip().lower()]


    for k, candidate in enumerate(candidates):
        print(f"     translation {k + 1}: ", candidate)
        score = evaluate_penalty(reference, candidate)
        print("         score:", score)

    candidates = []

REFERENCE:  itlanahnamic
     translation 1:  tlen motekiuiya
         REREFERENCE NGRAMS: Counter({'i': 1, 'tla': 1, 'nah': 1, 'namic': 1})
         CANDIDATE NGRAMS: Counter({'tlen': 1, 'mo': 1, 'te': 1, 'ki': 1, 'ui': 1, 'ya': 1})
         CORRECT PREDICTIONS:  0
         SUM MORPHEMES CANDIDATE:  6
         score: 0.0
     translation 2:  itlanajnamik
         REREFERENCE NGRAMS: Counter({'i': 1, 'tla': 1, 'nah': 1, 'namic': 1})
         CANDIDATE NGRAMS: Counter({'i': 1, 'tlan': 1, 'ajnamik': 1})
         CORRECT PREDICTIONS:  1
         SUM MORPHEMES CANDIDATE:  3
         score: 0.12262648039048077
     translation 3:  itlanajnamik
         REREFERENCE NGRAMS: Counter({'i': 1, 'tla': 1, 'nah': 1, 'namic': 1})
         CANDIDATE NGRAMS: Counter({'i': 1, 'tlan': 1, 'ajnamik': 1})
         CORRECT PREDICTIONS:  1
         SUM MORPHEMES CANDIDATE:  3
         score: 0.12262648039048077
     translation 4:  itlanajnamik
         REREFERENCE NGRAMS: Counter({'i': 1, 'tla': 1, 'nah': 1

In [ ]:
# Evaluate HISTORIAWITH brevity penalty
import re

file_path = 'refs_and_candids_historia.txt'
count = 1
candidates = []


with open(file_path, 'r') as file:
  lines = file.readlines()

  # in groups of 5
  for i in range(0, len(lines), 5):
    # saves first line as reference
    reference = lines[i].strip().lower()  # First line is reference
    print("REFERENCE: ", reference)

    # save lines 2-4 as candidates
    for j in range(i + 1, i + 5):
      candidates += [lines[j].strip().lower()]


    for k, candidate in enumerate(candidates):
        print(f"     translation {k + 1}: ", candidate)
        score = evaluate_penalty(reference, candidate)
        print("         score:", score)

    candidates = []

REFERENCE:  ipan matlactli omome tecpatl xihuitl oyaque mexicah tecpayocan
     translation 1:  ipan pedernal nopa mexikamej yajkej tecpayocan
         REREFERENCE NGRAMS: Counter({'tecpa': 2, 'ipan': 1, 'matlac': 1, 'tli': 1, 'om': 1, 'ome': 1, 'tl': 1, 'xihuitl': 1, 'oya': 1, 'que': 1, 'mexica': 1, 'h': 1, 'yocan': 1})
         CANDIDATE NGRAMS: Counter({'j': 2, 'ipan': 1, 'pedernal': 1, 'nopa': 1, 'mexika': 1, 'me': 1, 'ya': 1, 'kej': 1, 'tecpa': 1, 'yocan': 1})
         CORRECT PREDICTIONS:  3
         SUM MORPHEMES CANDIDATE:  11
         score: 0.08214387597605512
     translation 2:  ipan matlaktlomome tekpatl xiuitl oyake mexikaj tekpayokan
         REREFERENCE NGRAMS: Counter({'tecpa': 2, 'ipan': 1, 'matlac': 1, 'tli': 1, 'om': 1, 'ome': 1, 'tl': 1, 'xihuitl': 1, 'oya': 1, 'que': 1, 'mexica': 1, 'h': 1, 'yocan': 1})
         CANDIDATE NGRAMS: Counter({'te': 2, 'ipan': 1, 'ma': 1, 'tla': 1, 'ktlom': 1, 'ome': 1, 'k': 1, 'patl': 1, 'xiuitl': 1, 'oya': 1, 'ke': 1, 'mexika': 1, 'j